# Lake model continued

In the previous week you used the lake problem as a means of getting aquinted with the workbench. In this assignment we will continue with the lake problem, focussing explicitly on using it for open exploration. You can use the second part of [this tutorial](https://emaworkbench.readthedocs.io/en/latest/indepth_tutorial/open-exploration.html) for help.

**It is paramount that you are using the lake problem with 100 decision variables, rather than the one found on the website with the seperate anthropogenic release decision**

## Apply sensitivity analysis
There is substantial support in the ema_workbench for global sensitivity. For this, the workbench relies on [SALib](https://salib.readthedocs.io/en/latest/) and feature scoring which is a machine learning alternative for global sensitivity analysis. 


1. Apply Sobol with 3 seperate release policies (0, 0.05, and 0.1) and analyse the results for each release policy seperately focusing on the reliability objective. Do the sensitivities change depending on the release policy? Can you explain why or why not?

*hint: you can use sobol sampling for the uncertainties, and set policies to a list with the 3 different release policies. Next, for the analysis using logical indexing on the experiment.policy column you can select the results for each seperate release policy and apply sobol to each of the three seperate release policies. If this sounds too complicated, just do it on each release policy seperately.*



2. Repeat the above analysis for the 3 release policies but now with extra trees feature scoring and for all outcomes of interest. As a bonus, use the sobol experiment results as input for extra trees, and compare the results with those resulting from latin hypercube sampling.

*hint: you can use [seaborn heatmaps](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for a nice figure of the results*


In [1]:
from lakemodel_function import lake_problem

from ema_workbench import (Model, RealParameter, ScalarOutcome)

#instantiate the model
lake_model = Model('lakeproblem', function=lake_problem)
lake_model.time_horizon = 100 # used to specify the number of timesteps

#specify uncertainties
lake_model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

# set levers, one for each time step
lake_model.levers = [RealParameter(f"l{i}", 0, 0.1) for i in 
                     range(lake_model.time_horizon)] # we use time_horizon here

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P'),
                       ScalarOutcome('utility'),
                       ScalarOutcome('inertia'),
                       ScalarOutcome('reliability')]

In [2]:
from ema_workbench import Policy, perform_experiments
from ema_workbench import ema_logging

ema_logging.log_to_stderr(ema_logging.INFO)



<Logger EMA (DEBUG)>

In [3]:
 # add policies
policies = [Policy('no policy',
                      **{l.name:0 for l in lake_model.levers}),
            Policy('small release',
                       **{l.name:0.05 for l in lake_model.levers}),
            Policy('large release',
                       **{l.name:0.1 for l in lake_model.levers})
                ]

In [4]:
n_scenarios = 100
results = perform_experiments(lake_model, n_scenarios, policies=policies, uncertainty_sampling='sobol')
experiments, outcomes = results


[MainProcess/INFO] performing 1200 scenarios * 3 policies * 1 model(s) = 3600 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 360 cases completed
[MainProcess/INFO] 720 cases completed
[MainProcess/INFO] 1080 cases completed
[MainProcess/INFO] 1440 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 2160 cases completed
[MainProcess/INFO] 2520 cases completed
[MainProcess/INFO] 2880 cases completed
[MainProcess/INFO] 3240 cases completed
[MainProcess/INFO] 3600 cases completed
[MainProcess/INFO] experiments finished


In [12]:
reliability = outcomes['reliability']

In [6]:
no_policy_experiments = experiments[0:1200]
no_policy_reliability = reliability[0:1200]
no_policy_experiments

,b,delta,mean,q,stdev,l0,l1,l2,l3,l4,...,l93,l94,l95,l96,l97,l98,l99,scenario,policy,model
0,0.176904,0.935801,0.030742,3.691895,0.002121,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,no policy,lakeproblem
1,0.417529,0.935801,0.030742,3.691895,0.002121,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,no policy,lakeproblem
2,0.176904,0.932754,0.030742,3.691895,0.002121,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,no policy,lakeproblem
3,0.176904,0.935801,0.045977,3.691895,0.002121,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,no policy,lakeproblem
4,0.176904,0.935801,0.030742,3.252441,0.002121,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,no policy,lakeproblem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,0.354468,0.965244,0.012559,3.839600,0.003592,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1195,no policy,lakeproblem
1196,0.354468,0.939346,0.032598,3.839600,0.003592,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1196,no policy,lakeproblem
1197,0.354468,0.939346,0.012559,2.260010,0.003592,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1197,no policy,lakeproblem
1198,0.354468,0.939346,0.012559,3.839600,0.004885,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1198,no policy,lakeproblem


In [13]:
small_release_experiments = experiments[1200:2400]
small_release_reliability = reliability[1200:2400]
small_release_experiments

,b,delta,mean,q,stdev,l0,l1,l2,l3,l4,...,l93,l94,l95,l96,l97,l98,l99,scenario,policy,model
1200,0.176904,0.935801,0.030742,3.691895,0.002121,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0,small release,lakeproblem
1201,0.417529,0.935801,0.030742,3.691895,0.002121,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,1,small release,lakeproblem
1202,0.176904,0.932754,0.030742,3.691895,0.002121,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,2,small release,lakeproblem
1203,0.176904,0.935801,0.045977,3.691895,0.002121,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,3,small release,lakeproblem
1204,0.176904,0.935801,0.030742,3.252441,0.002121,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,4,small release,lakeproblem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,0.354468,0.965244,0.012559,3.839600,0.003592,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,1195,small release,lakeproblem
2396,0.354468,0.939346,0.032598,3.839600,0.003592,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,1196,small release,lakeproblem
2397,0.354468,0.939346,0.012559,2.260010,0.003592,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,1197,small release,lakeproblem
2398,0.354468,0.939346,0.012559,3.839600,0.004885,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,1198,small release,lakeproblem


In [14]:
large_release_experiments = experiments[2400:3600]
large_release_reliability = reliability[2400:3600]
large_release_experiments

,b,delta,mean,q,stdev,l0,l1,l2,l3,l4,...,l93,l94,l95,l96,l97,l98,l99,scenario,policy,model
2400,0.176904,0.935801,0.030742,3.691895,0.002121,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0,large release,lakeproblem
2401,0.417529,0.935801,0.030742,3.691895,0.002121,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1,large release,lakeproblem
2402,0.176904,0.932754,0.030742,3.691895,0.002121,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,2,large release,lakeproblem
2403,0.176904,0.935801,0.045977,3.691895,0.002121,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,3,large release,lakeproblem
2404,0.176904,0.935801,0.030742,3.252441,0.002121,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,4,large release,lakeproblem
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,0.354468,0.965244,0.012559,3.839600,0.003592,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1195,large release,lakeproblem
3596,0.354468,0.939346,0.032598,3.839600,0.003592,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1196,large release,lakeproblem
3597,0.354468,0.939346,0.012559,2.260010,0.003592,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1197,large release,lakeproblem
3598,0.354468,0.939346,0.012559,3.839600,0.004885,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,1198,large release,lakeproblem
